In [66]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

Import data from csv fies

In [67]:
ppi_dat = pd.read_csv('../data/STRING/STRING-interaction-swiss.csv')
ppi_dat.head()

,protein1_acc,protein2_acc,combined_score
0,P84085,Q14123,155
1,P84085,Q13177,197
2,P84085,O95755,222
3,P84085,Q13905,181
4,P84085,Q15057,767


In [68]:
proteins = pd.read_csv('../data/proteins-sample-0981.csv')
proteins.head()

,protein_accession,protein_score,protein_avg_mass,protein_matched_products,protein_matched_peptides,protein_digest_peps,protein_seq_cover_percent,protein_matched_peptide_inten_sum,protein_top3matched_peptide_inten_sum,protein_matched_product_inten_sum,protein_auto_curate,protein_sum_num_by_calc,protein_sum_num_by_pep_frag1,protein_num_unique_peptides
0,Q96EG1,0.0,57913.1781,0,1,33,0.0,2767,0.0,339,Green,5.981677,0,0
1,Q8TEX9,0.0,120633.6768,3,1,84,0.0,1469,0.0,488,Red,6.113683,0,0
2,Q0VDD8,0.0,405223.5318,10,3,268,0.0,13319,0.0,2544,Red,19.128590,0,0
3,Q9HCH5,0.0,105285.2734,1,3,85,0.0,130667,0.0,422,Green,28.732410,0,0
4,Q9BYN7,0.0,95001.7193,7,2,43,0.0,3633,0.0,1708,Red,7.312799,0,0


In [69]:
nextprot_dat = pd.read_csv('../data/nextprot-proteins.csv')
nextprot_dat.head()

,acc_code,protein_name,gene_name_s,chromosome,proteomics,disease,structure,number_isof,number_variants,number_ptms,mutagenesis,tissue_expr,pe
0,A0A024RBG1,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,1q21.2,yes,no,no,1,0,0,no,yes,Evidence at protein level
1,A0A075B6H7,Probable non-functional immunoglobulin kappa v...,IGKV3-7,2p11.2,yes,no,no,1,187,1,no,yes,Evidence at protein level
2,A0A075B6H8,Probable non-functional immunoglobulin kappa v...,IGKV1D-42,2p11.2,yes,no,no,1,140,1,no,yes,Evidence at protein level
3,A0A075B6H9,Immunoglobulin lambda variable 4-69,IGLV4-69,22q11.22,yes,no,no,1,318,1,no,yes,Evidence at protein level
4,A0A075B6I0,Immunoglobulin lambda variable 8-61,IGLV8-61,22q11.22,yes,no,no,1,256,1,no,yes,Evidence at protein level


In [70]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(nextprot_dat['acc_code'])
le.classes_

LabelEncoder()

array(['A0A024RBG1', 'A0A075B6H7', 'A0A075B6H8', ..., 'W5XKT8', 'W6CW81',
       'X6R8D5'], dtype=object)

Transform protein accession to integer ID

In [71]:
# proteins['acc_int_id'] = le.transform(proteins['protein_accession'])
# ppi_dat['protein1_acc_int_id'] = le.transform(ppi_dat['protein1_acc'])
# ppi_dat['protein2_acc_int_id'] = le.transform(ppi_dat['protein2_acc'])

## Proteins in PPI dataset but not in protein dataset
The protein dataset contain protein existence evidence information from an experiment (mostly  calculated protein identification features from MS spectrum)

The number of unique proteins in protein dataset and PPI dataset may not be the same.
We can either: 1. create those new protein entry in the protein dataset
or 2. only use PPI interaction information about proteins that exist in the protein dataset

In [72]:
# use approach 2 for simplicity
ppi_dat = ppi_dat[ppi_dat.protein1_acc.isin(proteins.protein_accession) & ppi_dat.protein2_acc.isin(proteins.protein_accession)]

# Create customized dataset

In [116]:
mapping = {index: i for i, index in enumerate(proteins.protein_accession.unique())}

In [117]:
import torch
node_attr = proteins.loc[:, ~proteins.columns.isin(['protein_accession', 'proteins.columns'])]

In [104]:
# need to define some helper encoder and transformer class

class LogTransformation(object):
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(np.log10(df.values+ 1)).view(-1, 1).to(self.dtype)

In [106]:
log_transformer = LogTransformation(dtype=torch.float)
x = log_transformer(node_attr.protein_score)
# also need train_masking, test_masking

In [61]:
src = [mapping[index] for index in ppi_dat.protein1_acc]
dst = [mapping[index] for index in ppi_dat.protein2_acc]
edge_index = torch.tensor([src, dst])
edge_index_reverse = torch.tensor([dst, src])

In [110]:
edge_attr = torch.tensor(ppi_dat.combined_score.values).view(-1, 1)

In [ ]:
from  torch_geometric.data import Data

In [112]:
# create graph dataset with node features, edge index, and edge attributes
dataset = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

In [114]:
print(f'Number of nodes: {dataset.num_nodes}')
print(f'Number of edges: {dataset.num_edges}')
print(f'Average node degree: {dataset.num_edges / dataset.num_nodes:.2f}')
print(f'Number of training nodes: {dataset.train_mask.sum()}')
print(f'Training node label rate: {int(dataset.train_mask.sum()) / dataset.num_nodes:.2f}')
print(f'Has isolated nodes: {dataset.has_isolated_nodes()}')
print(f'Has self-loops: {dataset.has_self_loops()}')
print(f'Is undirected: {dataset.is_undirected()}')

Number of nodes: 10628
Number of edges: 4938716
Average node degree: 464.69


AttributeError: 'GlobalStorage' object has no attribute 'train_mask'

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F


class MLP(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(dataset.num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

model = MLP(hidden_channels=16)
print(model)